In [65]:
import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 0) #2 titik perpuluhan


date = '20220831'
#jgn lupa rename nme file


date1 = "Aug'22"
GB = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\01_Corporate_Financing\01_Portfolio\\'+date+'\working - SH\GB Monthly Loan Outstanding '+date1+'.xlsx', sheet_name='SourceCM') 
#sheet_name='cb os by mkt seg 2'
#sheet_name='SourceCM'

description = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\Python\User defined\description.xlsx", sheet_name='description')

Report_Corp_CRDM = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\11_CRRS\\'+date[:4]+'\\'+date+'\Report_Corp_CRDM_'+date+'.xlsx')

#x install pyxlsb so xleh import 2007 punya excel, engine='pyxlsb'
date2 = 'Aug 2022'
CRWA = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\12_RDMS\\'+date+'\working - SH\CRWA '+date2+' Final (Domestic - All except Retail).xlsx', sheet_name='Context Id=162')
#sheet_name='Context Id=139'
#sheet_name='Context Id=162'

List = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\12_RDMS\\'+date+'\working - SH\List Master Listing '+date+'.xlsx')
#, sheet_name='Sheet2'

MasterBB = pd.read_excel (r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\01_Portfolio\\"+date+"\\Master BB @ "+date2+" Funded Non Funded Final to others.xlsx", sheet_name='Master Raw')

RSME = pd.read_excel (r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\03_Retail_SME_Financing\01_Portfolio\20220831\\"+date+"_SME Loan_v1_working_sent to MIB.xlsx", sheet_name='ALL w LMT')


In [68]:
#=================================================CB=======================================================================

GB.columns = GB.columns.str.strip()
GB.columns = GB.columns.str.upper()
GB.columns = GB.columns.str.replace(" ","_")

GB.M_FINANCING_TYPE_CD = GB.M_FINANCING_TYPE_CD.str.strip()
GB.M_FINANCING_TYPE_CD = GB.M_FINANCING_TYPE_CD.str.upper()

GB.FACILITY = GB.FACILITY.str.strip()
GB.FACILITY = GB.FACILITY.str.upper()

#case 20211231,20220331,20210930 nme M_BNM_BALANCE_SUM_1
#GB['M_BNM_BALANCE_SUM']=GB['M_BNM_BALANCE_SUM_1']

#M_BNM_BALANCE_SUM 20220731
GB.M_BNM_BALANCE_SUM.fillna(0, inplace=True)


GB1 = GB.iloc[np.where((GB.M_FINANCING_TYPE_CD=='I')&(GB.FACILITY!='RTO'))]

#case 20220831, 20210930
GB1['M_CUS_NO'] = GB1['M_CUS_NO'].astype('Int64').astype('str')

#case 20211231
#check M_MSIC_CODE kat excel n let type as general sbb klau x dia jd time value

GB1=GB1[['M_CUS_NO','NAME','M_MSIC_CODE','M_BNM_BALANCE_SUM']]

GB1 = GB1.fillna(0).groupby(['M_CUS_NO','NAME','M_MSIC_CODE'])[['M_BNM_BALANCE_SUM']].sum().reset_index().sort_values(by='M_BNM_BALANCE_SUM', ascending=False)


description.columns = description.columns.str.strip()
description.columns = description.columns.str.replace(" ", "_")
description.columns = description.columns.str.upper()

description.ISC_NOB = description.ISC_NOB.map(str)
description.ISC_NOB = description.ISC_NOB.str.strip()
description.ISC_NOB = description.ISC_NOB.str.upper()

GB1.M_MSIC_CODE = GB1.M_MSIC_CODE.map(str)
GB1.M_MSIC_CODE = GB1.M_MSIC_CODE.str.strip()
GB1.M_MSIC_CODE = GB1.M_MSIC_CODE.str.upper()

join_A = GB1.merge(description[['ISC_NOB','INDUSTRIAL_SECTOR_CODE_(ISC_DESC)2','ISC_PARENT']].drop_duplicates().rename(columns = {'ISC_NOB':'M_MSIC_CODE'}), on='M_MSIC_CODE', how='left').drop('M_BNM_BALANCE_SUM', axis=1)


Report_Corp_CRDM.columns = Report_Corp_CRDM.columns.str.strip()
Report_Corp_CRDM.columns = Report_Corp_CRDM.columns.str.upper()
Report_Corp_CRDM.columns = Report_Corp_CRDM.columns.str.replace(" ","_")

Report_Corp_CRDM1 = Report_Corp_CRDM[['GCIF_NUM','RDMS_ADJ_BRR']]

Report_Corp_CRDM1.GCIF_NUM = Report_Corp_CRDM1.GCIF_NUM.map(str)
Report_Corp_CRDM1.GCIF_NUM = Report_Corp_CRDM1.GCIF_NUM.str.strip()
Report_Corp_CRDM1.GCIF_NUM = Report_Corp_CRDM1.GCIF_NUM.str.upper()

join_A.M_CUS_NO = join_A.M_CUS_NO.map(str)
join_A.M_CUS_NO = join_A.M_CUS_NO.str.strip()
join_A.M_CUS_NO = join_A.M_CUS_NO.str.upper()

#case 20210630
#join_A['M_CUS_NO'] = join_A['M_CUS_NO'].str[:-2]

join_B = join_A.merge(Report_Corp_CRDM1.rename(columns = {'GCIF_NUM':'M_CUS_NO'}).drop_duplicates(), on='M_CUS_NO', how='left')




CRWA.columns = CRWA.columns.str.strip()
CRWA.columns = CRWA.columns.str.upper()
CRWA.columns = CRWA.columns.str.replace(" ","_")

CRWA1 = CRWA[['COUNTERPARTY_ORIGINAL','CUSTOMER_FIELD19','INTERNAL_RATING_ORIGINAL']]

CRWA1.COUNTERPARTY_ORIGINAL = CRWA1.COUNTERPARTY_ORIGINAL.map(str)
CRWA1.COUNTERPARTY_ORIGINAL = CRWA1.COUNTERPARTY_ORIGINAL.str.strip()
CRWA1.COUNTERPARTY_ORIGINAL = CRWA1.COUNTERPARTY_ORIGINAL.str.upper()

join_C = join_B.merge(CRWA1.rename(columns = {'COUNTERPARTY_ORIGINAL':'M_CUS_NO'}).drop_duplicates('M_CUS_NO'), on='M_CUS_NO', how='left').drop_duplicates()


def PSE_SOV(CUSTOMER_FIELD19):
    if CUSTOMER_FIELD19 == 'PSE(SA)':
        return "PSE"
    elif CUSTOMER_FIELD19 == 'SOV(SA)':
        return "SOV"
#    else:
#        return "Unrated"
join_C['PSE/SOV'] = join_C.CUSTOMER_FIELD19.apply(PSE_SOV)

List.columns = List.columns.str.strip()
List.columns = List.columns.str.upper()
List.columns = List.columns.str.replace(" ","_")

List.GCIF_CHILD = List.GCIF_CHILD.map(str)
List.GCIF_CHILD = List.GCIF_CHILD.str.strip()
List.GCIF_CHILD = List.GCIF_CHILD.str.upper()

join_D = join_C.merge(List.rename(columns={"GCIF_CHILD":"M_CUS_NO"}).drop_duplicates(), on='M_CUS_NO', how='left').drop_duplicates()


#join_D.GCIF_PARENTS = join_D.GCIF_PARENTS.apply(lambda x: f'{x:.0f}')
join_D.GCIF_PARENTS = join_D.GCIF_PARENTS.astype(str)
join_D.GCIF_PARENTS = join_D.GCIF_PARENTS.str.strip()
join_D.GCIF_PARENTS = join_D.GCIF_PARENTS.str.upper()

join_D['GCIF_PARENTS1'] = join_D.GCIF_PARENTS.str[:-2]


join_E = join_D.merge(join_D[['M_CUS_NO','INTERNAL_RATING_ORIGINAL']].rename(columns={'M_CUS_NO':'GCIF_PARENTS1','INTERNAL_RATING_ORIGINAL':'3RD_PARTY'}), on='GCIF_PARENTS1', how='left')


join_E1 = join_E.merge(CRWA1.rename(columns = {'COUNTERPARTY_ORIGINAL':'GCIF_PARENTS1'}).drop_duplicates('GCIF_PARENTS1'), on='GCIF_PARENTS1', how='left',suffixes=('', '_y')).drop_duplicates()

join_E1['CUSTOMER_FIELD19'] = join_E1.CUSTOMER_FIELD19
join_E1.loc[join_E1['CUSTOMER_FIELD19'].isnull(),'CUSTOMER_FIELD19'] = join_E1.CUSTOMER_FIELD19_y


join_E1['3RD_PARTY'] = join_E1['3RD_PARTY']
join_E1.loc[join_E1['3RD_PARTY'].isnull(),'3RD_PARTY'] = join_E1.INTERNAL_RATING_ORIGINAL_y


join_E2 = join_E1.merge(Report_Corp_CRDM1.rename(columns = {'GCIF_NUM':'GCIF_PARENTS1'}).drop_duplicates('GCIF_PARENTS1'), on='GCIF_PARENTS1', how='left',suffixes=('', '_y')).drop_duplicates()

join_E2['3RD_PARTY'] = join_E2['3RD_PARTY']
join_E2.loc[join_E2['3RD_PARTY'].isnull(),'3RD_PARTY'] = join_E2.RDMS_ADJ_BRR_y


join_E2['FINAL_RATING'] = join_E2['INTERNAL_RATING_ORIGINAL']
join_E2.loc[join_E2['FINAL_RATING'].isnull(),'FINAL_RATING'] = join_E2['3RD_PARTY']
join_E2.loc[join_E2['FINAL_RATING'].isnull(),'FINAL_RATING'] = join_E2.RDMS_ADJ_BRR
join_E2.loc[join_E2['FINAL_RATING'].isnull(),'FINAL_RATING'] = join_E2['PSE/SOV']
join_E2.loc[join_E2['FINAL_RATING'].isnull(),'FINAL_RATING'] = "Unrated"


join_E2.rename(columns={'M_MSIC_CODE':'BNM_SECTOR_CD',\
                       'INDUSTRIAL_SECTOR_CODE_(ISC_DESC)2':'SECTOR_DESC',\
                      'ISC_PARENT':'SECTOR_PARENT',\
                      'RDMS_ADJ_BRR':'CRRS',\
                      'INTERNAL_RATING_ORIGINAL':'RDMS',\
                      'GCIF_PARENTS1':'3RD_PARTY_PARENT'},inplace=True)

join_E2['SEGMENT'] = '01. CB'

join_E2 = join_E2[['M_CUS_NO',\
                'NAME',\
                'BNM_SECTOR_CD',\
                'SECTOR_DESC',\
                'SECTOR_PARENT',\
                'SEGMENT',\
                'RDMS',\
                'CUSTOMER_FIELD19',\
                'PSE/SOV',\
                'CRRS',\
                '3RD_PARTY_PARENT',\
                '3RD_PARTY',\
                'FINAL_RATING']]

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [107]:
#=================================================BB=======================================================================

#case 20220831
MasterBB['Balance'] = MasterBB['O/S @Aug 2022']

MasterBB1 = MasterBB.iloc[np.where((MasterBB.MBB_Islamic_Ind=='I'))].sort_values(by='Balance', ascending=False)[['GCIF','Org_Name','MISC_Cd','MISC_Desc','BRR']]
#&(MasterBB.Facility_Level_2!='Non-Funded Tradebills')&(MasterBB.Facility_Level_2!='FEC (10%)')

MasterBB1['GCIF'] = MasterBB1['GCIF'].astype('Int64').astype('str')

MasterBB1['SEGMENT']='02. BB'

MasterBB1['MISC_Cd'] = MasterBB1['MISC_Cd'].map(str)
MasterBB1['MISC_Cd'] = MasterBB1['MISC_Cd'].str.strip()
MasterBB1['MISC_Cd'] = MasterBB1['MISC_Cd'].str.upper()
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['01120','0113X','01140','0119XA','0119XB','01261A','01261B','01262A','01262B','01263','01269','01273','01279','01291','01299','0130X','0141X','0144X','0145X','0146XA','0146XB','0146XC','0149X','01500','016XX','0170X','0210X','0220X','0230X','0240X','031XXA','031XXB','031XXC','031XXD','032XXA','032XXB','032XXC','1120','113X','1140','119XA','119XB','1261A','1261B','1262A','1262B','1263','1269','1273','1279','1291','1299','130X','141X','144X','145X','146XA','146XB','146XC','149X','1500','16XX','170X','210X','220X','230X','240X','31XXA','31XXB','31XXC','31XXD','32XXA','32XXB','32XXC']),'ISC_PARENT1'] = "01. Agriculture, Forestry and Fishing"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['05XXX','0610X','0620X','0710X','07210','07291','07299','08XXX','0910XA','0910XB','0910XC','0910XD','0910XE','0910XF','0910XG','09900', '5XXX','610X','620X','710X','7210','7291','7299','8XXX','910XA','910XB','910XC','910XD','910XE','910XF','910XG','9900']),'ISC_PARENT1'] = "02. Mining and Quarrying"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['1010X','1020X','1030X','10401A','10401B','10402A','10402B','10499','1050X','106XX','107XX','10800','110XXA','110XXB','12000','13XXX','14XXX','151XX','1520X','16XXX','1701X','181XX','18200','19XXXA','19XXXB','20XXX','21XXX','22XXXA','22XXXB','22XXXC','22XXXD','22XXXE','22XXXF','22XXXG','23XXXA','23XXXB','23XXXC','2410XA','2410XB','24201','24202','24209A','24209B','2431XA','2431XB','24320','251XX','25200','25910','25920','25930','2599X','26300','265XX','2670X','26999','27500','27999','28170','2819XA','2819XB','2819XC','2819XD','2819XE','28220','28299A','28299B','28299C','28299D','28299E','29XXXA','29XXXB','29XXXC','301XXA','301XXB','301XXC','301XXD','30200','30999A','30999B','30999C','3100X','32500','3290X','33XXXA','33XXXB','33XXXC','33XXXD','33XXXE','33XXXF','33XXXG','33XXXH']),'ISC_PARENT1'] =  "03. Manufacturing";
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['3510XA','3510XB','3510XC','3510XD','3510XE','3510XF','3510XG','3520XA','3520XB','3530X']),'ISC_PARENT1'] = "04. Electricity, Gas, Steam and Air Conditioning Supply"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['360XX','3700X','38112','38199','382XXA','382XXB','382XXC','3830X','39000']),'ISC_PARENT1']="05. Water Supply; Sewerage, Waste Management and Remediation Activities"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['41001A','41001B','41001C','41001D','41001E','41001F','41001G','41001H','41002A','41002B','41002C','41002D','41002E','41002F','41002G','41002H','41002I','41002J','41003','41009','4210XA','4210XB','4210XC','4210XD','4210XE','4210XF','4210XG','4220XA','4220XB','4220XC','4220XD','429XXA','429XXB','429XXC','431XX','432XX','4330X','4390X']),'ISC_PARENT1'] = "06. Construction"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['451XXA','451XXB','451XXC','451XXD','451XXE','4520X','4530XA','4530XB','4540X','462XXA','462XXB','462XXC','463XXA','463XXB','463XXC','463XXD','4641X','46496','46499','46510','46595','46599A','46599B','46599C','46999','4711X','4719X','472XXA','472XXB','472XXC','472XXD','478XX','479XX','40799A','40799B','40799C','40799D','40799E']),'ISC_PARENT1'] = "07. Wholesale and Retail Trade; Repair of Motor Vehicles and Motorcycles"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['4911X','4912X','4921X','4922X','49230','49300','5011X','50121','50122','5021X','5022X','51101A','51101B','51102','51103','51201','51202','51203','5210XA','5210XB','5221X','5222X','5223X','52241','52249','52291','52292','52299','53XXX']),'ISC_PARENT1'] = "08. Transportation and Storage"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['55101A','55101B','55101C','55101D','55104','55105','55108','55199','5520X','55900','56101','56102','56103','56104','56105','56106','56107','562XX','56302','56303','56399A','56399B']),'ISC_PARENT1'] = "09. Accommodation and Food Service Activities"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['58110','5812X','5813X','5819X','582XX','59XXX','60XXX','6110X','6120X','6130X','6190X','620XX','631XX','6391X','63990']),'ISC_PARENT1'] = "10. Information and Communication"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['64110','64191','64192','64193','64194','64195','64199','64200','64301','64302','64303','64304','64309','64910','64921','64922','64923','64924','64925','64929','64991','64992','64993','64999','651XX','6520X','65301','65302','66111','66112','66113','66114','66119','66121','66122','66123','66124','66125','66129','66191','66192','66199','66211','66212','66221','66222','66223','66224','66290','66301','66302','66303']),'ISC_PARENT1'] = "11. Financial and Insurance/ Takaful Activities"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['68101A','68101B','68101C','68102A','68102B','68102C','68102D','68102E','68103A','68103B','68104A','68104B','68104C','68104D','68104E','68104F','68104G','68109','68201','68202','68203','68209']),'ISC_PARENT1'] =  "12. Real Estate Activities"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['69XXX','70100','70201','70202','70203','70209','7110X','71200','72102','72103','72105','72106','72199','7220X','73100','73200','74XXX','75000']),'ISC_PARENT1'] =  "13. Professional, Scientific and Technical Activities"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['7710X','772XX','77301','77307','77399','77400','78XXX','79XXX','80XXX','81XXX','821XX','82200','8230X','82910','8292XA','8292XB','8292XC','82990']),'ISC_PARENT1'] = "14. Administrative and Support Service Activities"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['841XX','842XX','8430X']),'ISC_PARENT1'] = "15. Public Administration and Defence; Compulsory Social Security"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['8510X','852XX','8530X','854XX','8550X']),'ISC_PARENT1'] = "16. Education"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['861XX','8620X','8690X','87XXX','88XXX']),'ISC_PARENT1'] = "17. Human Health and Social Work Activities"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['9000X','910XX','92000','93XXX']),'ISC_PARENT1'] = "18. Arts, Entertainment and Recreation"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['94XXXA','94XXXB','95XXX','960XX']),'ISC_PARENT1'] = "19. Other Service Activities"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['TXXXX']),'ISC_PARENT1'] =  "20. Activities of Households as Employers; Undifferentiated Goods and Services Producing Activities of Households for Own Use"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['99000']),'ISC_PARENT1'] = "21. Activities of Extraterritorial Organisations and Bodies"
MasterBB1.loc[MasterBB1.MISC_Cd.isin(['97000']),'ISC_PARENT1'] = "22. Household"
MasterBB1.ISC_PARENT1.fillna(-9999, inplace=True)
MasterBB1.loc[MasterBB1.MISC_Cd.isin([-9999]),'ISC_PARENT1'] = "23. Not Populated"


MasterBB1['RDMS'] = ''
MasterBB1['CUSTOMER_FIELD19'] = ''
MasterBB1['PSE/SOV'] = ''
MasterBB1['CRRS'] = ''
MasterBB1['3RD_PARTY_PARENT'] = ''
MasterBB1['3RD_PARTY'] = ''

MasterBB1.ISC_PARENT1 = MasterBB1.ISC_PARENT1.str.upper()

MasterBB2 = MasterBB1.rename(columns = {'MISC_Cd':'BNM_SECTOR_CD',\
                   'GCIF':'M_CUS_NO',\
                   'Org_Name':'NAME',\
                   'BRR':'FINAL_RATING',\
                   'ISC_PARENT1':'SECTOR_PARENT',\
                   'MISC_Desc':'SECTOR_DESC'}).groupby(['M_CUS_NO','NAME','BNM_SECTOR_CD','SECTOR_DESC','SECTOR_PARENT','SEGMENT','RDMS','CUSTOMER_FIELD19','PSE/SOV','CRRS','3RD_PARTY_PARENT','3RD_PARTY','FINAL_RATING']).sum().reset_index()

In [148]:
RSME['M Entity Cd'] = RSME['M Entity Cd'].str.strip()

RSME1  = RSME.iloc[np.where((RSME['M Entity Cd']=='MIB'))].sort_values(by='M Bnm Balance', ascending=False)[['M Cus No','Name','M MSIC CODE','MSIC Group','Risk Grade Overall (FINAL)']]

RSME1['M Cus No'] = RSME1['M Cus No'].astype('Int64').astype('str')

RSME1['SEGMENT'] = '03. SME'

RSME1['M MSIC CODE'] = RSME1['M MSIC CODE'].map(str)
RSME1['M MSIC CODE'] = RSME1['M MSIC CODE'].str.strip()
RSME1['M MSIC CODE'] = RSME1['M MSIC CODE'].str.upper()

RSME1.rename(columns={'M MSIC CODE':'MISC_Cd'},inplace=True)

RSME1.loc[RSME1.MISC_Cd.isin(['01120','0113X','01140','0119XA','0119XB','01261A','01261B','01262A','01262B','01263','01269','01273','01279','01291','01299','0130X','0141X','0144X','0145X','0146XA','0146XB','0146XC','0149X','01500','016XX','0170X','0210X','0220X','0230X','0240X','031XXA','031XXB','031XXC','031XXD','032XXA','032XXB','032XXC','1120','113X','1140','119XA','119XB','1261A','1261B','1262A','1262B','1263','1269','1273','1279','1291','1299','130X','141X','144X','145X','146XA','146XB','146XC','149X','1500','16XX','170X','210X','220X','230X','240X','31XXA','31XXB','31XXC','31XXD','32XXA','32XXB','32XXC']),'ISC_PARENT1'] = "01. Agriculture, Forestry and Fishing"
RSME1.loc[RSME1.MISC_Cd.isin(['05XXX','0610X','0620X','0710X','07210','07291','07299','08XXX','0910XA','0910XB','0910XC','0910XD','0910XE','0910XF','0910XG','09900', '5XXX','610X','620X','710X','7210','7291','7299','8XXX','910XA','910XB','910XC','910XD','910XE','910XF','910XG','9900']),'ISC_PARENT1'] = "02. Mining and Quarrying"
RSME1.loc[RSME1.MISC_Cd.isin(['1010X','1020X','1030X','10401A','10401B','10402A','10402B','10499','1050X','106XX','107XX','10800','110XXA','110XXB','12000','13XXX','14XXX','151XX','1520X','16XXX','1701X','181XX','18200','19XXXA','19XXXB','20XXX','21XXX','22XXXA','22XXXB','22XXXC','22XXXD','22XXXE','22XXXF','22XXXG','23XXXA','23XXXB','23XXXC','2410XA','2410XB','24201','24202','24209A','24209B','2431XA','2431XB','24320','251XX','25200','25910','25920','25930','2599X','26300','265XX','2670X','26999','27500','27999','28170','2819XA','2819XB','2819XC','2819XD','2819XE','28220','28299A','28299B','28299C','28299D','28299E','29XXXA','29XXXB','29XXXC','301XXA','301XXB','301XXC','301XXD','30200','30999A','30999B','30999C','3100X','32500','3290X','33XXXA','33XXXB','33XXXC','33XXXD','33XXXE','33XXXF','33XXXG','33XXXH']),'ISC_PARENT1'] =  "03. Manufacturing";
RSME1.loc[RSME1.MISC_Cd.isin(['3510XA','3510XB','3510XC','3510XD','3510XE','3510XF','3510XG','3520XA','3520XB','3530X']),'ISC_PARENT1'] = "04. Electricity, Gas, Steam and Air Conditioning Supply"
RSME1.loc[RSME1.MISC_Cd.isin(['360XX','3700X','38112','38199','382XXA','382XXB','382XXC','3830X','39000']),'ISC_PARENT1']="05. Water Supply; Sewerage, Waste Management and Remediation Activities"
RSME1.loc[RSME1.MISC_Cd.isin(['41001A','41001B','41001C','41001D','41001E','41001F','41001G','41001H','41002A','41002B','41002C','41002D','41002E','41002F','41002G','41002H','41002I','41002J','41003','41009','4210XA','4210XB','4210XC','4210XD','4210XE','4210XF','4210XG','4220XA','4220XB','4220XC','4220XD','429XXA','429XXB','429XXC','431XX','432XX','4330X','4390X']),'ISC_PARENT1'] = "06. Construction"
RSME1.loc[RSME1.MISC_Cd.isin(['451XXA','451XXB','451XXC','451XXD','451XXE','4520X','4530XA','4530XB','4540X','462XXA','462XXB','462XXC','463XXA','463XXB','463XXC','463XXD','4641X','46496','46499','46510','46595','46599A','46599B','46599C','46999','4711X','4719X','472XXA','472XXB','472XXC','472XXD','478XX','479XX','40799A','40799B','40799C','40799D','40799E']),'ISC_PARENT1'] = "07. Wholesale and Retail Trade; Repair of Motor Vehicles and Motorcycles"
RSME1.loc[RSME1.MISC_Cd.isin(['4911X','4912X','4921X','4922X','49230','49300','5011X','50121','50122','5021X','5022X','51101A','51101B','51102','51103','51201','51202','51203','5210XA','5210XB','5221X','5222X','5223X','52241','52249','52291','52292','52299','53XXX']),'ISC_PARENT1'] = "08. Transportation and Storage"
RSME1.loc[RSME1.MISC_Cd.isin(['55101A','55101B','55101C','55101D','55104','55105','55108','55199','5520X','55900','56101','56102','56103','56104','56105','56106','56107','562XX','56302','56303','56399A','56399B']),'ISC_PARENT1'] = "09. Accommodation and Food Service Activities"
RSME1.loc[RSME1.MISC_Cd.isin(['58110','5812X','5813X','5819X','582XX','59XXX','60XXX','6110X','6120X','6130X','6190X','620XX','631XX','6391X','63990']),'ISC_PARENT1'] = "10. Information and Communication"
RSME1.loc[RSME1.MISC_Cd.isin(['64110','64191','64192','64193','64194','64195','64199','64200','64301','64302','64303','64304','64309','64910','64921','64922','64923','64924','64925','64929','64991','64992','64993','64999','651XX','6520X','65301','65302','66111','66112','66113','66114','66119','66121','66122','66123','66124','66125','66129','66191','66192','66199','66211','66212','66221','66222','66223','66224','66290','66301','66302','66303']),'ISC_PARENT1'] = "11. Financial and Insurance/ Takaful Activities"
RSME1.loc[RSME1.MISC_Cd.isin(['68101A','68101B','68101C','68102A','68102B','68102C','68102D','68102E','68103A','68103B','68104A','68104B','68104C','68104D','68104E','68104F','68104G','68109','68201','68202','68203','68209']),'ISC_PARENT1'] =  "12. Real Estate Activities"
RSME1.loc[RSME1.MISC_Cd.isin(['69XXX','70100','70201','70202','70203','70209','7110X','71200','72102','72103','72105','72106','72199','7220X','73100','73200','74XXX','75000']),'ISC_PARENT1'] =  "13. Professional, Scientific and Technical Activities"
RSME1.loc[RSME1.MISC_Cd.isin(['7710X','772XX','77301','77307','77399','77400','78XXX','79XXX','80XXX','81XXX','821XX','82200','8230X','82910','8292XA','8292XB','8292XC','82990']),'ISC_PARENT1'] = "14. Administrative and Support Service Activities"
RSME1.loc[RSME1.MISC_Cd.isin(['841XX','842XX','8430X']),'ISC_PARENT1'] = "15. Public Administration and Defence; Compulsory Social Security"
RSME1.loc[RSME1.MISC_Cd.isin(['8510X','852XX','8530X','854XX','8550X']),'ISC_PARENT1'] = "16. Education"
RSME1.loc[RSME1.MISC_Cd.isin(['861XX','8620X','8690X','87XXX','88XXX']),'ISC_PARENT1'] = "17. Human Health and Social Work Activities"
RSME1.loc[RSME1.MISC_Cd.isin(['9000X','910XX','92000','93XXX']),'ISC_PARENT1'] = "18. Arts, Entertainment and Recreation"
RSME1.loc[RSME1.MISC_Cd.isin(['94XXXA','94XXXB','95XXX','960XX']),'ISC_PARENT1'] = "19. Other Service Activities"
RSME1.loc[RSME1.MISC_Cd.isin(['TXXXX']),'ISC_PARENT1'] =  "20. Activities of Households as Employers; Undifferentiated Goods and Services Producing Activities of Households for Own Use"
RSME1.loc[RSME1.MISC_Cd.isin(['99000']),'ISC_PARENT1'] = "21. Activities of Extraterritorial Organisations and Bodies"
RSME1.loc[RSME1.MISC_Cd.isin(['97000']),'ISC_PARENT1'] = "22. Household"
RSME1.ISC_PARENT1.fillna(-9999, inplace=True)
RSME1.loc[RSME1.MISC_Cd.isin([-9999]),'ISC_PARENT1'] = "23. Not Populated"


RSME1['RDMS'] = ''
RSME1['CUSTOMER_FIELD19'] = ''
RSME1['PSE/SOV'] = ''
RSME1['CRRS'] = ''
RSME1['3RD_PARTY_PARENT'] = ''
RSME1['3RD_PARTY'] = ''

RSME1.ISC_PARENT1 = RSME1.ISC_PARENT1.str.upper()

RSME2 = RSME1.rename(columns = {'MISC_Cd':'BNM_SECTOR_CD',\
                   'M Cus No':'M_CUS_NO',\
                   'Name':'NAME',\
                   'Risk Grade Overall (FINAL)':'FINAL_RATING',\
                   'ISC_PARENT1':'SECTOR_PARENT',\
                   'MSIC Group':'SECTOR_DESC'})[['M_CUS_NO','NAME','BNM_SECTOR_CD','SECTOR_DESC','SECTOR_PARENT','SEGMENT','RDMS','CUSTOMER_FIELD19','PSE/SOV','CRRS','3RD_PARTY_PARENT','3RD_PARTY','FINAL_RATING']].drop_duplicates()

In [158]:
frames = [join_E2, MasterBB2, RSME2] 
Rating_Combine = pd.concat(frames)

In [33]:


writer2 = pd.ExcelWriter(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\18_Rating_CBBBRSME\BRR_CBBBRSME_'+date[:6]+'(python-new rules).xlsx',engine='xlsxwriter')

join_E2.to_excel(writer2, index = False)

writer2.save()

In [ ]:
=='Unrated' !='N'

In [72]:
a = join_E2.iloc[np.where(join_E2['FINAL_RATING']=='Unrated')]
a.shape

(50, 13)

In [73]:
b = MasterBB2.iloc[np.where(MasterBB2['FINAL_RATING']=='Unrated')]
b.shape

(16, 13)

In [155]:
c = RSME2.iloc[np.where(RSME2['FINAL_RATING']=='Unrated')]
c.shape

(0, 13)

# FIX SECTION

In [13]:
Nisa = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\18_Rating_CBBBRSME\BRR_CBBBRSME_202208.xlsx', sheet_name='Sheet1', header=0)

Nisa.columns = Nisa.columns.str.replace(" ","_").str.upper()
Nisa1 = Nisa.iloc[np.where(Nisa.SEGMENT=='01. CB')][['M_CUS_NO','FINAL_RATING']]

In [20]:
Nisa['M_CUS_NO'] = Nisa['M_CUS_NO'].astype('Int64').astype('str')


In [55]:
CEK = join_E2.merge(Nisa[['M_CUS_NO','FINAL_RATING']],on='M_CUS_NO',how='left',indicator=True, suffixes=('','_y'))

In [56]:
Nisa1.shape, join_E.shape, CEK.shape

((551, 2), (518, 12), (518, 15))

In [65]:
#CEK._merge.value_counts()

In [64]:
y = CEK.iloc[np.where(CEK.FINAL_RATING_y!='Unrated')]
#y.iloc[np.where(y.FINAL_RATING=='Unrated')]

In [63]:
#CEK.iloc[np.where(CEK.M_CUS_NO.isin(['7051644','2182543','27777772','27777242']))]

# RDMS

In [77]:
Rating = pd.read_excel (r"T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\18_Rating_CBBBRSME\BRR_CBBBRSME_"+date[:6]+".xlsx", sheet_name='Sheet1',header=0)
Rating.M_CUS_NO = Rating.M_CUS_NO.astype(str)

In [160]:
Rating_Combine.SEGMENT.value_counts()

03. SME    39659
02. BB      3977
01. CB       551
Name: SEGMENT, dtype: int64

In [153]:
MasterBB2.shape, join_E2.shape, RSME2.shape

((3977, 13), (551, 13), (39659, 13))

In [154]:
Rating.SEGMENT.value_counts()

03. SME    39659
02. BB      3977
01. CB       551
Name: SEGMENT, dtype: int64

In [157]:
CRWA.head(1)

,PARTITION_KEY,DEALBOOK,COUNTERPARTY_ORIGINAL,CPTY_ORI_DESC,COUNTERPARTY_NEW,TABLE_NAME,FAMILY,CUSTOMER_FIELD18,CUSTOMER_FIELD19,CUSTOMER_FIELD20,EXPOSURE_ATTRIBUTE_6,BS_TYPE,COUNTERPARTY_NEW_ATTRIBUTE_6,COLLATERAL_BIS_PRODUCT,TRANCHE_COLLATERAL_TYPE,TRANCHE_GUARANTEE_TYPE,METHOD_ORIGINAL,INT_RATING_AGENCY_ORIGINAL,OBLIGOR_GRADE_RANK_ORIGINAL,INTERNAL_RATING_ORIGINAL,CUSTOMER_FIELD17,EXPOSURE_ATTRIBUTE_1,COUNTERPARTY_TYPE_ORIGINAL,PD_ORIGINAL,LGD,METHOD_NEW,PD_NEW,INT_RATING_AGENCY_NEW,INTERNAL_RATING_NEW,OBLIGOR_GRADE_RANK_NEW,EXPOSURE_ATTRIBUTE_4,INDUSTRY_SECTOR_ORIGINAL,RISK_COUNTRY_NEW,INCORPORATION_COUNTRY_NEW,CURRENCY,VALUE_DATE,TRADE_DATE,MATURITY_DATE,MATURITY_EFF,RW_NEW,EXPOSURE_ATTRIBUTE_7,COUNTERPARTY_NEW_ATTRIBUTE_3,COUNTERPARTY_NEW_ATTRIBUTE_5,EAD,RWA_BEFORE_SCALING,RWA_AFTER_SCALING,EXPECTED_LOSS,CAPITAL_REQUIREMENT,SUM(NOMINAL),SUM(OUTSTANDING),SUM(AVAILABLE_AMOUNT),SUM(FAIR_VALUE),SUM(ACCRUED_INTEREST),CPTY_NEW_DESC,ASSET_CLASS_ORIGINAL,ASSET_CLASS_NEW,BIS_PAST_DUE,COUNTERPARTY_NEW_ATTRIBUTE_4,K_MATURITY,PERFORMING_STATUS,INCORPORATION_COUNTRY_ORIGINAL,GUARANTOR_BIS_ENTITY_TYPE,SUM(EAD_PRO_PRE_CCF),SUM(CUSTOMER_NUMBER18),CCF,COUNTRY_OF_RESIDENCE
0,202208310802,MBB_MY,16087928,EVER-ADVANCE SDN. BHD.,16087928,ON_OFF_BALANCE,BA,BB_TB,CORP SME(FIRB),M_BA_OWN,MBBM MM BA PRIMARY,ASS,1_2_1,NaN,NOT COLLATERALIZED,NOT GUARANTEED,FOU,ZMB_CORP_L,1,10,NaN,NaN,104,5e-03,5e-01,FOU,5e-03,ZMB_CORP_L,10,1,120100150111,M_4719X,MY,MY,MYR,2022-06-28,2022-06-28,2022-10-26 00:00:00,1,4e-01,M_0470,MEDIUM-SIZED COMPANIES,7,408547,177874,188546,1002,15084,41,408547,,,,EVER-ADVANCE SDN. BHD.,CLAIM_SME,CLAIM_SME,F,NaN,1,1,MY,NaN,,1,1,MY


In [219]:
CRWA1 = CRWA.sort_values(by='SUM(OUTSTANDING)', ascending=False)[['COUNTERPARTY_ORIGINAL','CPTY_ORI_DESC','DEALBOOK','INTERNAL_RATING_ORIGINAL','INDUSTRY_SECTOR_ORIGINAL']].drop_duplicates()

RDMS_Only = CRWA1.rename(columns={'COUNTERPARTY_ORIGINAL':'M_CUS_NO'}).merge(Rating_Combine[['M_CUS_NO']], on='M_CUS_NO',how='left', indicator=True)

RDMS_Only1 = RDMS_Only.iloc[np.where(RDMS_Only._merge=='left_only')].drop('_merge', axis=1)

RDMS_Only1['MISC_Cd'] = RDMS_Only1.INDUSTRY_SECTOR_ORIGINAL.str.lstrip("M_")

#RDMS_Only1['MISC_Cd'] = RDMS_Only1['MISC_Cd'].astype('Int64').astype('str')

RDMS_Only1['SEGMENT'] = '04. RDMS'

RDMS_Only1['MISC_Cd'] = RDMS_Only1['MISC_Cd'].map(str)
RDMS_Only1['MISC_Cd'] = RDMS_Only1['MISC_Cd'].str.strip()
RDMS_Only1['MISC_Cd'] = RDMS_Only1['MISC_Cd'].str.upper()

RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['01120','0113X','01140','0119XA','0119XB','01261A','01261B','01262A','01262B','01263','01269','01273','01279','01291','01299','0130X','0141X','0144X','0145X','0146XA','0146XB','0146XC','0149X','01500','016XX','0170X','0210X','0220X','0230X','0240X','031XXA','031XXB','031XXC','031XXD','032XXA','032XXB','032XXC','1120','113X','1140','119XA','119XB','1261A','1261B','1262A','1262B','1263','1269','1273','1279','1291','1299','130X','141X','144X','145X','146XA','146XB','146XC','149X','1500','16XX','170X','210X','220X','230X','240X','31XXA','31XXB','31XXC','31XXD','32XXA','32XXB','32XXC']),'ISC_PARENT1'] = "01. Agriculture, Forestry and Fishing"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['05XXX','0610X','0620X','0710X','07210','07291','07299','08XXX','0910XA','0910XB','0910XC','0910XD','0910XE','0910XF','0910XG','09900', '5XXX','610X','620X','710X','7210','7291','7299','8XXX','910XA','910XB','910XC','910XD','910XE','910XF','910XG','9900']),'ISC_PARENT1'] = "02. Mining and Quarrying"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['1010X','1020X','1030X','10401A','10401B','10402A','10402B','10499','1050X','106XX','107XX','10800','110XXA','110XXB','12000','13XXX','14XXX','151XX','1520X','16XXX','1701X','181XX','18200','19XXXA','19XXXB','20XXX','21XXX','22XXXA','22XXXB','22XXXC','22XXXD','22XXXE','22XXXF','22XXXG','23XXXA','23XXXB','23XXXC','2410XA','2410XB','24201','24202','24209A','24209B','2431XA','2431XB','24320','251XX','25200','25910','25920','25930','2599X','26300','265XX','2670X','26999','27500','27999','28170','2819XA','2819XB','2819XC','2819XD','2819XE','28220','28299A','28299B','28299C','28299D','28299E','29XXXA','29XXXB','29XXXC','301XXA','301XXB','301XXC','301XXD','30200','30999A','30999B','30999C','3100X','32500','3290X','33XXXA','33XXXB','33XXXC','33XXXD','33XXXE','33XXXF','33XXXG','33XXXH']),'ISC_PARENT1'] =  "03. Manufacturing";
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['3510XA','3510XB','3510XC','3510XD','3510XE','3510XF','3510XG','3520XA','3520XB','3530X']),'ISC_PARENT1'] = "04. Electricity, Gas, Steam and Air Conditioning Supply"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['360XX','3700X','38112','38199','382XXA','382XXB','382XXC','3830X','39000']),'ISC_PARENT1']="05. Water Supply; Sewerage, Waste Management and Remediation Activities"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['41001A','41001B','41001C','41001D','41001E','41001F','41001G','41001H','41002A','41002B','41002C','41002D','41002E','41002F','41002G','41002H','41002I','41002J','41003','41009','4210XA','4210XB','4210XC','4210XD','4210XE','4210XF','4210XG','4220XA','4220XB','4220XC','4220XD','429XXA','429XXB','429XXC','431XX','432XX','4330X','4390X']),'ISC_PARENT1'] = "06. Construction"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['451XXA','451XXB','451XXC','451XXD','451XXE','4520X','4530XA','4530XB','4540X','462XXA','462XXB','462XXC','463XXA','463XXB','463XXC','463XXD','4641X','46496','46499','46510','46595','46599A','46599B','46599C','46999','4711X','4719X','472XXA','472XXB','472XXC','472XXD','478XX','479XX','40799A','40799B','40799C','40799D','40799E']),'ISC_PARENT1'] = "07. Wholesale and Retail Trade; Repair of Motor Vehicles and Motorcycles"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['4911X','4912X','4921X','4922X','49230','49300','5011X','50121','50122','5021X','5022X','51101A','51101B','51102','51103','51201','51202','51203','5210XA','5210XB','5221X','5222X','5223X','52241','52249','52291','52292','52299','53XXX']),'ISC_PARENT1'] = "08. Transportation and Storage"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['55101A','55101B','55101C','55101D','55104','55105','55108','55199','5520X','55900','56101','56102','56103','56104','56105','56106','56107','562XX','56302','56303','56399A','56399B']),'ISC_PARENT1'] = "09. Accommodation and Food Service Activities"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['58110','5812X','5813X','5819X','582XX','59XXX','60XXX','6110X','6120X','6130X','6190X','620XX','631XX','6391X','63990']),'ISC_PARENT1'] = "10. Information and Communication"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['64110','64191','64192','64193','64194','64195','64199','64200','64301','64302','64303','64304','64309','64910','64921','64922','64923','64924','64925','64929','64991','64992','64993','64999','651XX','6520X','65301','65302','66111','66112','66113','66114','66119','66121','66122','66123','66124','66125','66129','66191','66192','66199','66211','66212','66221','66222','66223','66224','66290','66301','66302','66303']),'ISC_PARENT1'] = "11. Financial and Insurance/ Takaful Activities"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['68101A','68101B','68101C','68102A','68102B','68102C','68102D','68102E','68103A','68103B','68104A','68104B','68104C','68104D','68104E','68104F','68104G','68109','68201','68202','68203','68209']),'ISC_PARENT1'] =  "12. Real Estate Activities"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['69XXX','70100','70201','70202','70203','70209','7110X','71200','72102','72103','72105','72106','72199','7220X','73100','73200','74XXX','75000']),'ISC_PARENT1'] =  "13. Professional, Scientific and Technical Activities"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['7710X','772XX','77301','77307','77399','77400','78XXX','79XXX','80XXX','81XXX','821XX','82200','8230X','82910','8292XA','8292XB','8292XC','82990']),'ISC_PARENT1'] = "14. Administrative and Support Service Activities"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['841XX','842XX','8430X']),'ISC_PARENT1'] = "15. Public Administration and Defence; Compulsory Social Security"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['8510X','852XX','8530X','854XX','8550X']),'ISC_PARENT1'] = "16. Education"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['861XX','8620X','8690X','87XXX','88XXX']),'ISC_PARENT1'] = "17. Human Health and Social Work Activities"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['9000X','910XX','92000','93XXX']),'ISC_PARENT1'] = "18. Arts, Entertainment and Recreation"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['94XXXA','94XXXB','95XXX','960XX']),'ISC_PARENT1'] = "19. Other Service Activities"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['TXXXX']),'ISC_PARENT1'] =  "20. Activities of Households as Employers; Undifferentiated Goods and Services Producing Activities of Households for Own Use"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['99000']),'ISC_PARENT1'] = "21. Activities of Extraterritorial Organisations and Bodies"
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin(['97000']),'ISC_PARENT1'] = "22. Household"
RDMS_Only1.ISC_PARENT1.fillna(-9999, inplace=True)
RDMS_Only1.loc[RDMS_Only1.MISC_Cd.isin([-9999]),'ISC_PARENT1'] = "23. Not Populated"

RDMS_Only1['RDMS'] = ''
RDMS_Only1['CUSTOMER_FIELD19'] = ''
RDMS_Only1['PSE/SOV'] = ''
RDMS_Only1['CRRS'] = ''
RDMS_Only1['3RD_PARTY_PARENT'] = ''
RDMS_Only1['3RD_PARTY'] = ''
RDMS_Only1['SECTOR_DESC'] = ''

RDMS_Only1.ISC_PARENT1 = RDMS_Only1.ISC_PARENT1.str.upper()

RDMS_Only2 = RDMS_Only1.rename(columns = {'MISC_Cd':'BNM_SECTOR_CD',\
                   'M_CUS_NO':'M_CUS_NO',\
                   'CPTY_ORI_DESC':'NAME',\
                   'INTERNAL_RATING_ORIGINAL':'FINAL_RATING',\
                   'ISC_PARENT1':'SECTOR_PARENT'})[['M_CUS_NO','NAME','BNM_SECTOR_CD','SECTOR_DESC','SECTOR_PARENT','SEGMENT','RDMS','CUSTOMER_FIELD19','PSE/SOV','CRRS','3RD_PARTY_PARENT','3RD_PARTY','FINAL_RATING']].drop_duplicates()

In [220]:
RDMS_Only2

,M_CUS_NO,NAME,BNM_SECTOR_CD,SECTOR_DESC,SECTOR_PARENT,SEGMENT,RDMS,CUSTOMER_FIELD19,PSE/SOV,CRRS,3RD_PARTY_PARENT,3RD_PARTY,FINAL_RATING
0,8236272,BANK NEGARA MALAYSIA,64110,,11. FINANCIAL AND INSURANCE/ TAKAFUL ACTIVITIES,04. RDMS,,,,,,,NaN
2,15388614,MAYBANK,64910,,11. FINANCIAL AND INSURANCE/ TAKAFUL ACTIVITIES,04. RDMS,,,,,,,8
5,17879819,GOV OF THE UNITED STATES,841XX,,15. PUBLIC ADMINISTRATION AND DEFENCE; COMPULSORY SOCIAL SECURITY,04. RDMS,,,,,,,NaN
8,1503027,AMBANK (M) BERHAD,64304,,11. FINANCIAL AND INSURANCE/ TAKAFUL ACTIVITIES,04. RDMS,,,,,,,8
10,17218431,MAYBANK ISLAMIC BERHAD,64192,,11. FINANCIAL AND INSURANCE/ TAKAFUL ACTIVITIES,04. RDMS,,,,,,,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
137864,16598080,FUNNEL MALAYSIA SDN.BHD FKA PACIFC,620XX,,10. INFORMATION AND COMMUNICATION,04. RDMS,,,,,,,21
137866,NaN,NaN,97000,,22. HOUSEHOLD,04. RDMS,,,,,,,NaN
137867,2664624,SUPREME HOUSING SDN BHD,41001F,,06. CONSTRUCTION,04. RDMS,,,,,,,NaN
137868,20012906,HSBC HOLDINGS PLC,66199,,11. FINANCIAL AND INSURANCE/ TAKAFUL ACTIVITIES,04. RDMS,,,,,,,17


In [214]:
Rating_Combine.head(1)

,M_CUS_NO,NAME,BNM_SECTOR_CD,SECTOR_DESC,SECTOR_PARENT,SEGMENT,RDMS,CUSTOMER_FIELD19,PSE/SOV,CRRS,3RD_PARTY_PARENT,3RD_PARTY,FINAL_RATING
0,23549043,LEMBAGA PEMBIAYAAN PERUMAHAN SEKTOR AWAM,64999,"OTHER FINANCIAL SERVICE ACTIVITIES, EXCEPT INSURANCE/TAKAFUL AND PENSION FUNDING N.E.C.",11. FINANCIAL AND INSURANCE/ TAKAFUL ACTIVITIES,01. CB,2,PSE(FIRB),None,2,N,NaN,2


In [203]:
intro = "<>I'm Tom."
intro = intro.lstrip("<>")